In [3]:
# example model to run for now just to get an idea 
import torch
import torch.nn as nn
import pandas as pd

# trialing a pipeline 

Here we want to create a trial pipleine , in this trial pipeline we will have random integers the goal is to get out model to output some scores for a couple pipes basically for now they can be random integers 

In [2]:
# example model to run for now just to get an idea 
import torch
import torch.nn as nn


class TowerOne(nn.Module):
    def __init__(self):
        super(TowerOne, self).__init__()
        
        # First layer: input 64 features, output 32 features
        self.fc1 = nn.Linear(64, 32)
        
        # Second layer: input 32 features, output 16 features
        self.fc2 = nn.Linear(32, 16)
        
        # Third layer: input 16 features, output 3 features (final output)
        self.fc3 = nn.Linear(16, 3)
        
        # Activation function (ReLU) applied after first and second layers
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass input through the first layer, followed by ReLU
        x = self.relu(self.fc1(x))
        
        # Pass through the second layer, followed by ReLU
        x = self.relu(self.fc2(x))
        
        # Pass through the final layer to get 3 output features
        x = self.fc3(x)
        
        return x

class TowerTwo(nn.Module):
    def __init__(self):
        super(TowerTwo, self).__init__()
        self.fc = nn.Linear(64, 3)
    def forward(self, x):
        x = self.fc(x)
        return x


In [14]:
 
#call embedder


# Generate random inputs
video = torch.rand(1, 64)
video = torch.rand(1, 64)
video = torch.rand(1, 64)
video = torch.rand(1, 64)


query = torch.rand(1, 3)   # Input for TowerTwo

# Initialize models
tower_one = TowerOne()
tower_two = TowerTwo()

# Forward pass through both towers
output_one = tower_one(video)  # Pass video through TowerOne
output_two = tower_two(query)  # Pass query through TowerTwo

# Compute cosine similarity between the outputs
score = nn.functional.cosine_similarity(output_one, output_two, dim=1)

# Define target and compute loss (mean squared error)
target = torch.tensor([1.0])  # The target cosine similarity
loss = (score - target).pow(2).mean()

# Backpropagation
loss.backward()

# Print outputs for debugging
print("Video input:", video)
print("Query input:", query)
print("Output from TowerOne:", output_one)
print("Output from TowerTwo:", output_two)
print("Cosine similarity score:", score)
print("Loss:", loss)


Video input: tensor([[0.4139, 0.7380, 0.2999, 0.3325, 0.0293, 0.4814, 0.3391, 0.9069, 0.7763,
         0.0498, 0.5203, 0.0422, 0.3766, 0.6510, 0.6568, 0.4529, 0.9977, 0.2522,
         0.9066, 0.0753, 0.8177, 0.1161, 0.6997, 0.4761, 0.5777, 0.1587, 0.9614,
         0.5546, 0.4156, 0.5668, 0.9692, 0.1272, 0.5269, 0.8872, 0.6158, 0.8411,
         0.7299, 0.1364, 0.8546, 0.0254, 0.3160, 0.8907, 0.3916, 0.0663, 0.2259,
         0.9908, 0.0894, 0.6401, 0.2493, 0.8152, 0.6204, 0.2751, 0.6881, 0.4009,
         0.7173, 0.4172, 0.1987, 0.7279, 0.9791, 0.4464, 0.2679, 0.2666, 0.5608,
         0.2206]])
Query input: tensor([[0.2647, 0.4808, 0.9866]])
Output from TowerOne: tensor([[-0.0077,  0.0032, -0.2654]], grad_fn=<AddmmBackward0>)
Output from TowerTwo: tensor([[-0.4474,  0.2700,  0.4150]], grad_fn=<AddmmBackward0>)
Cosine similarity score: tensor([-0.5973], grad_fn=<SumBackward1>)
Loss: tensor(2.5513, grad_fn=<MeanBackward0>)


In [18]:
import torch
import torch.nn as nn

# Define margin for triplet loss
margin = 1.0


positive_video1 = torch.rand(1, 64)
positive_video2 = torch.rand(1, 64)
negative_video1 = torch.rand(1, 64)
negative_video2 = torch.rand(1, 64)

anchor_query = torch.rand(1, 64)  # Input for TowerTwo (optional for your task)

# Initialize models
tower_one = TowerOne()
tower_two = TowerTwo()

# Forward pass through the tower for each video
anchor_output = tower_one(anchor_query)          # Anchor
positive_output1 = tower_two(positive_video1)    # Positive sample 1
positive_output2 = tower_two(positive_video2)    # Positive sample 2
negative_output1 = tower_two(negative_video1)    # Negative sample 1
negative_output2 = tower_two(negative_video2)    # Negative sample 2

# Cosine similarity between anchor and positives
cosine_sim_positive1 = nn.functional.cosine_similarity(anchor_output, positive_output1, dim=1)
cosine_sim_positive2 = nn.functional.cosine_similarity(anchor_output, positive_output2, dim=1)


# Cosine similarity between anchor and negatives
cosine_sim_negative1 = nn.functional.cosine_similarity(anchor_output, negative_output1, dim=1)
cosine_sim_negative2 = nn.functional.cosine_similarity(anchor_output, negative_output2, dim=1)

# Compute Triplet Loss (for positive1-negative1 and positive2-negative2)
triplet_loss1 = torch.clamp(margin - cosine_sim_positive1 + cosine_sim_negative1, min=0)
triplet_loss2 = torch.clamp(margin - cosine_sim_positive2 + cosine_sim_negative2, min=0)

# Average the losses
triplet_loss = (triplet_loss1 + triplet_loss2).mean()

# Backpropagation
triplet_loss.backward()

# Print debugging information
print("Anchor output:", anchor_output)
print("Positive 1 output:", positive_output1)
print("Positive 2 output:", positive_output2)
print("Negative 1 output:", negative_output1)
print("Negative 2 output:", negative_output2)
print("Cosine similarity positive 1:", cosine_sim_positive1)
print("Cosine similarity positive 2:", cosine_sim_positive2)
print("Cosine similarity negative 1:", cosine_sim_negative1)
print("Cosine similarity negative 2:", cosine_sim_negative2)
print("Triplet loss:", triplet_loss)


Anchor output: tensor([[-0.1423, -0.1539,  0.1909]], grad_fn=<AddmmBackward0>)
Positive 1 output: tensor([[-0.2249,  0.2396,  0.2570]], grad_fn=<AddmmBackward0>)
Positive 2 output: tensor([[-0.6124,  0.3555,  0.0086]], grad_fn=<AddmmBackward0>)
Negative 1 output: tensor([[-0.4607,  0.0852,  0.0928]], grad_fn=<AddmmBackward0>)
Negative 2 output: tensor([[-0.3622,  0.3388,  0.0395]], grad_fn=<AddmmBackward0>)
Cosine similarity positive 1: tensor([0.3736], grad_fn=<SumBackward1>)
Cosine similarity positive 2: tensor([0.1697], grad_fn=<SumBackward1>)
Cosine similarity negative 1: tensor([0.5182], grad_fn=<SumBackward1>)
Cosine similarity negative 2: tensor([0.0493], grad_fn=<SumBackward1>)
Triplet loss: tensor(2.0242, grad_fn=<MeanBackward0>)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for progress bar

# Define the margin for triplet loss
margin = 1.0
num_epochs = 1000  # Number of iterations/epochs to run
learning_rate = 0.001  # Learning rate for optimizer
batch_size = 16  # Batch size for training

# Initialize models (Make sure TowerOne and TowerTwo are defined)
tower_one = TowerOne()
tower_two = TowerTwo()

# Define optimizer
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=learning_rate)

# TripletMarginWithDistanceLoss with cosine similarity as the distance function
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=margin
)

# Training loop with tqdm for progress tracking
with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as pbar:
    for epoch in range(num_epochs):
        # Generate random batches of video inputs (batch size anchor, 2 positive, 2 negative)
        anchor_query = torch.rand(batch_size, 64)    # Anchor batch
        positive_video1 = torch.rand(batch_size, 64) # Positive batch 1
        positive_video2 = torch.rand(batch_size, 64) # Positive batch 2
        negative_video1 = torch.rand(batch_size, 64) # Negative batch 1
        negative_video2 = torch.rand(batch_size, 64) # Negative batch 2
        
        # Zero gradients (reset gradients before backpropagation)
        optimizer.zero_grad()

        # Forward pass through the tower for each video batch
        anchor_output = tower_one(anchor_query)          # Anchor
        positive_output1 = tower_two(positive_video1)    # Positive sample 1
        positive_output2 = tower_two(positive_video2)    # Positive sample 2
        negative_output1 = tower_two(negative_video1)    # Negative sample 1
        negative_output2 = tower_two(negative_video2)    # Negative sample 2

        # Calculate triplet loss using custom cosine similarity distance
        triplet_loss1 = triplet_loss_fn(anchor_output, positive_output1, negative_output1)
        triplet_loss2 = triplet_loss_fn(anchor_output, positive_output2, negative_output2)

        # Average the triplet losses
        triplet_loss = (triplet_loss1 + triplet_loss2) / 2

        # Backpropagation
        triplet_loss.backward()

        # Update the model parameters
        optimizer.step()

        # Update tqdm description with current loss
        pbar.set_postfix({"Triplet Loss": triplet_loss.item()})
        pbar.update(1)  # Increment the progress bar

print("Training complete.")


Training Progress: 100%|██████████| 1000/1000 [00:02<00:00, 423.17epoch/s, Triplet Loss=1.04]


Training complete.


# trying out model 

In [1]:
# example model to run for now just to get an idea 
import torch
import torch.nn as nn
import pandas as pd

In [2]:
class SkipGramFoo(torch.nn.Module):
  def __init__(self, voc, emb, ctx):
    super().__init__()
    self.ctx = ctx
    self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
    self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
    self.sig = torch.nn.Sigmoid()

  # def forward(self, inpt, trgs, rand):
  #   emb = self.emb(inpt)
  #   ctx = self.ffw.weight[trgs]
  #   rnd = self.ffw.weight[rand]
  #   out = torch.mm(ctx, emb.T)
  #   rnd = torch.mm(rnd, emb.T)
  #   out = self.sig(out).clamp(min=1e-7, max=1 - 1e-7)
  #   rnd = self.sig(rnd).clamp(min=1e-7, max=1 - 1e-7)
  #   pst = -out.log().mean()
  #   ngt = -(1 - rnd).log().mean()
  #   return pst + ngt

#new forwarding for batch size 
  def forward(self, inpt, trgs, rand):
    # Embedding lookup for input (shape: [batch_size, embedding_dim])
    emb = self.emb(inpt)
    
    # Ensure context (trgs) and random samples (rand) have the same batch size as inpt
    batch_size = inpt.size(0)  # Get the current batch size

    # Slice or generate the random tensor according to the input batch size
    rand = rand[:batch_size]  # Adjust random tensor to match current batch size
    
    ctx = self.ffw.weight[trgs.to(inpt.device)]  # Shape: [batch_size, 2, embedding_dim]
    rnd = self.ffw.weight[rand.to(inpt.device)]  # Shape: [batch_size, 2, embedding_dim]

    # Ensure the batch size matches before performing batch matrix multiplication
    assert ctx.size(0) == emb.size(0), f"Context batch size {ctx.size(0)} doesn't match embeddings batch size {emb.size(0)}"
    assert rnd.size(0) == emb.size(0), f"Random batch size {rnd.size(0)} doesn't match embeddings batch size {emb.size(0)}"
    
    # Perform batch matrix multiplication
    out = torch.bmm(ctx, emb.unsqueeze(2)).squeeze(2)  # Shape: (batch_size, 2)
    rnd = torch.bmm(rnd, emb.unsqueeze(2)).squeeze(2)  # Shape: (batch_size, 2)
    
    # Apply sigmoid and clamp to prevent NaNs
    out = self.sig(out).clamp(min=1e-7, max=1 - 1e-7)
    rnd = self.sig(rnd).clamp(min=1e-7, max=1 - 1e-7)

    # Calculate loss
    pst = -out.log().mean()   # Positive sample log-likelihood
    ngt = -(1 - rnd).log().mean()  # Negative sample log-likelihood
    
    return pst + ngt

In [5]:
import torch

# Create an instance of your model
model_path = "C:/Users/omare/Desktop/hackernews_predicting_views/data processing/skipgram_model_titles.pth"
mfoo = torch.load(model_path, map_location=torch.device('cpu'))
mfoo = SkipGramFoo(voc=70572, emb=64, ctx=2)  # Initialize with proper params
mfoo.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
mfoo.eval()

C:\Users\omare\AppData\Local\Temp\ipykernel_26928\4031393700.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mfoo = torch.load(model_path, map_location=torch.device('cpu

SkipGramFoo(
  (emb): Embedding(70572, 64)
  (ffw): Linear(in_features=64, out_features=70572, bias=False)
  (sig): Sigmoid()
)

In [6]:
#loading in dataset of passages to add to dictionary 
extra_voc = pd.read_csv('extra_dict.csv')
new_words = extra_voc['passage_text']

In [7]:
import collections
import re

def preprocess(text: str) -> list[str]:
    # Convert text to lowercase
    text = text.lower()
    
    # Replace punctuation with placeholders
    text = text.replace('.',  ' <PERIOD> ')
    text = text.replace(',',  ' <COMMA> ')
    text = text.replace('"',  ' <QUOTATION_MARK> ')
    text = text.replace(';',  ' <SEMICOLON> ')
    text = text.replace('!',  ' <EXCLAMATION_MARK> ')
    text = text.replace('?',  ' <QUESTION_MARK> ')
    text = text.replace('(',  ' <LEFT_PAREN> ')
    text = text.replace(')',  ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace(':',  ' <COLON> ')

    # Split text into words
    words = text.split()
    
    # Count occurrences of each word
    stats = collections.Counter(words)
    
    # Filter words to keep only those that occur more than 15 times
    words = [word for word in words if stats[word] > 15]
    
    # Remove any remaining symbols (non-alphabetic characters)
    words = [word for word in words if re.match(r'^[a-zA-Z<>\s]+$', word)]
    
    return words

# Convert the Pandas Series to a list
new_words = new_words.tolist()  

# Now call your preprocess function
new_words = preprocess(' '.join(new_words))


In [8]:
import re
import collections

def identify_symbols(text) -> list[str]:
    # If text is a list, join it into a single string
    if isinstance(text, list):
        text = ' '.join(text)
    
    # Use regex to find non-alphabetic characters
    symbols = re.findall(r'[^a-zA-Z\s]', text)
    
    # Count the occurrences of each symbol
    symbol_counts = collections.Counter(symbols)
    
    return symbol_counts


# Identify symbols in the list of words
symbols = identify_symbols(new_words)

print("Symbols found:", symbols)



Symbols found: Counter({'<': 6075090, '>': 6075090})


In [10]:

# Load existing vocabulary
import pickle

with open("vocab_dict.pkl", "rb") as f:
    vocab_dict = pickle.load(f)

print(len(vocab_dict))

for word in new_words:
    if word not in vocab_dict:
        vocab_dict[word] = len(vocab_dict)  # Assign new index

print(len(vocab_dict))

70572
121598


In [11]:
from collections import defaultdict

# Invert vocab_dict to detect duplicates (word -> list of IDs)
inverted_vocab = defaultdict(list)

# Fill the inverted dictionary (key: word, value: list of IDs)
for word_id, word in vocab_dict.items():
    inverted_vocab[word].append(word_id)

# Check for duplicated words
duplicated_words = {word: ids for word, ids in inverted_vocab.items() if len(ids) > 1}

# Display the duplicated words and their IDs
if duplicated_words:
    print("Duplicated Words Found:")
    for word, ids in duplicated_words.items():
        print(f"Word: '{word}' has multiple IDs: {ids}")
else:
    print("No duplicated words found.")


No duplicated words found.


In [12]:

# Create a list of token IDs corresponding to the new words
new_token_ids = []

for word in new_words:
    if word in vocab_dict:
        new_token_ids.append(vocab_dict[word])  # Get the ID from the vocabulary dictionary
    else:
        print(f"Warning: '{word}' not found in the vocabulary dictionary.")  # Optional: handle missing words


In [15]:
# Check sizes
print(f"Old Vocabulary Size: {old_vocab_size}")
print(f"New Vocabulary Size: {new_vocab_size}")


Old Vocabulary Size: 70572
New Vocabulary Size: 121598


In [14]:
import torch

# Assuming mfoo is already defined and is your SkipGram model

# Get old vocabulary size and embedding dimension
old_vocab_size = 70572  # Your old vocab size
new_vocab_size = 121598 # Your new vocab size
embedding_dim = mfoo.emb.embedding_dim  # Get the embedding dimension

# Create a new embedding layer with the new vocabulary size
new_embedding_layer = torch.nn.Embedding(new_vocab_size, embedding_dim)

# Copy old weights to the new layer, ensuring you do not exceed the size
if old_vocab_size <= new_vocab_size:
    new_embedding_layer.weight.data[:old_vocab_size] = mfoo.emb.weight.data[:old_vocab_size]

# Initialize the new weights for the newly added words (if desired)
new_embedding_layer.weight.data[old_vocab_size:] = torch.randn(new_vocab_size - old_vocab_size, embedding_dim)

# Replace the old embedding layer with the new one
mfoo.emb = new_embedding_layer

# Update the Linear Layer (self.ffw) to handle the new vocabulary size
new_ffw_layer = torch.nn.Linear(in_features=embedding_dim, out_features=new_vocab_size, bias=False)

# Copy the old weights to the new linear layer
new_ffw_layer.weight.data[:old_vocab_size] = mfoo.ffw.weight.data[:old_vocab_size]

# Initialize the new weights for the new vocabulary
new_ffw_layer.weight.data[old_vocab_size:] = torch.randn(new_vocab_size - old_vocab_size, embedding_dim)

# Replace the old Linear layer (self.ffw) with the new one
mfoo.ffw = new_ffw_layer

# Now your model (mfoo) is updated and ready for fine-tuning with the new vocabulary size



In [18]:
import torch
import more_itertools
import tqdm
import wandb

# Initialize W&B
wandb.init(project="skipgram_training", entity="omareweis123", name='fine_tuning_skipgram/final/10ep')

# Set parameters
batch_size = 3000
learning_rate = 0.001 # Define your learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mFoo = mfoo.to(device)

# Set context size
context_size = 2  # Example context size
window_size = 2 * context_size + 1  # Total tokens in the window

# Initialize the optimizer
opFoo = torch.optim.Adam(mFoo.parameters(), lr=learning_rate)



# Create your new dataset (make sure tokens are prepared)
# Assume new_tokens is your new dataset's tokenized input
wins = list(more_itertools.windowed(new_token_ids, window_size))  # Create sliding windows for the new dataset

# Training loop
for epoch in range(10):
    prgs = tqdm.tqdm(range(0, len(wins), batch_size), total=len(wins) // batch_size, desc=f"Epoch {epoch + 1}", leave=False)

    total_loss = 0.0  # Initialize total loss for the epoch
    num_batches = 0   # Counter for batches

    for batch_idx in prgs:
        batch_wins = wins[batch_idx:batch_idx + batch_size]

        # Prepare batch inputs and targets
        inpts = torch.LongTensor([win[context_size] for win in batch_wins]).to(device)  # Central token for each window
        trgs = torch.LongTensor([[win[i] for i in range(context_size)] + [win[i] for i in range(context_size + 1, window_size)]
                                  for win in batch_wins]).to(device)  # Context tokens (left and right)
        rand = torch.randint(0, len(vocab_dict), (batch_size, 2)).to(device)  # Random negative samples

        # Zero gradients
        opFoo.zero_grad()

        # Forward pass
        loss = mFoo(inpts, trgs, rand)

        # Backward pass and optimization
        loss.backward()
        opFoo.step()

        # Accumulate loss
        total_loss += loss.item()
        num_batches += 1

        # Log the loss
        wandb.log({'loss': loss.item(), 'learning_rate': learning_rate})

    # Calculate and log average loss for the epoch
    average_loss = total_loss / num_batches if num_batches > 0 else 0
    wandb.log({'average_loss': average_loss})

# Finish the W&B logging
wandb.finish()


average_loss,█▂▂▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▅▄▄▃▃▂▂▃▂▂▂▂▂▂▂▁▁▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
average_loss,0.30825
learning_rate,0.001
loss,0.27672


average_loss,█▆▅▄▃▃▂▂▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▆▃█▆▄▄▇▅▄▇▄▆█▅▃▆▅▄▅▆▆█▄▆▁▃▂▆▅▆█▄█▄▅▂▅▄▆▅
average_loss,0.26334
learning_rate,0.001
loss,0.24762


In [38]:
import torch

# Define the file path where you want to save the model
model_save_path = "finetuned_skipgram_model.pth"

# Save entire model
torch.save({
    'model_state_dict': mFoo.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, "finetuned_skipgram_model.pth")



# Running the two tower model on the finetune embedder 

In [24]:
import torch
import torch.nn as nn

class TowerOneRNN(nn.Module):
    def __init__(self):
        super(TowerOneRNN, self).__init__()
        
        # Single LSTM layer: input 64 features, hidden size 32
        self.rnn = nn.RNN(input_size=64, hidden_size=32, batch_first=True)
        
        # Fully connected layer to map from 32 features to 3 (final output)
        self.fc = nn.Linear(32, 3)

    def forward(self, x):
        # Pass input through the LSTM layer
        x, _ = self.rnn(x)
        
        # The LSTM outputs sequences; get the output from the last time step
        x = x[:, -1, :]  # Keep the output for the last time step
        
        # Pass through the fully connected layer
        x = self.fc(x)
        
        return x

class TowerTwoRNN(nn.Module):
    def __init__(self):
        super(TowerTwoRNN, self).__init__()  # Corrected to use TowerTwoRNN
        
        # Single LSTM layer: input 64 features, hidden size 32
        self.rnn = nn.RNN(input_size=64, hidden_size=32, batch_first=True)
        
        # Fully connected layer to map from 32 features to 3 (final output)
        self.fc = nn.Linear(32, 3)

    def forward(self, x):
        # Pass input through the LSTM layer
        x, _ = self.rnn(x)
        
        # The LSTM outputs sequences; get the output from the last time step
        x = x[:, -1, :]  # Keep the output for the last time step
        
        # Pass through the fully connected layer
        x = self.fc(x)
        
        return x


In [21]:
results = pd.read_csv('results.csv')

In [39]:
import pandas as pd
import torch

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume `updated_vocab` is your vocabulary dictionary
updated_vocab = vocab_dict  # Replace with your actual vocab dictionary

# Step 1: Load the dataframe
valid_titles_df = pd.read_csv('extra_dict.csv')  # Assuming the file is valid_titles.csv
titles = valid_titles_df['query'].tolist()

# Function to create a reverse vocabulary from the updated vocabulary
def create_reverse_vocab(vocab):
    return {word: index for index, word in vocab.items()}

# Load the reverse vocabulary
reverse_vocab = create_reverse_vocab(updated_vocab)

# Tokenize the titles using the reverse vocabulary
def tokenize_titles(titles, reverse_vocab):
    tokens = []
    
    for title in titles:
        words = title.lower().split()  # Convert the title to lowercase to match training preprocessing
        
        tokenized = []
        for word in words:
            if word in reverse_vocab:
                tokenized.append(reverse_vocab[word])  # Get the index from reverse_vocab
            # No else clause needed; unknown words are simply skipped
        
        tokens.append(tokenized)
    
    return tokens

# Now we use the reverse_vocab to tokenize the titles
tokenized_titles = tokenize_titles(titles, reverse_vocab)

# Step 3: Define the SkipGramFoo model
class SkipGramFoo(torch.nn.Module):
    def __init__(self, voc, emb, ctx):
        super().__init__()
        self.ctx = ctx
        self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
        self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
        self.sig = torch.nn.Sigmoid()

    def forward(self, inpt, trgs, rand):
        emb = self.emb(inpt)
        batch_size = inpt.size(0)  # Get the current batch size
        rand = rand[:batch_size]  # Adjust random tensor to match current batch size
        
        ctx = self.ffw.weight[trgs.to(inpt.device)]  # Shape: [batch_size, embedding_dim]
        rnd = self.ffw.weight[rand.to(inpt.device)]  # Shape: [batch_size, embedding_dim]

        # Perform batch matrix multiplication
        out = torch.bmm(ctx.view(batch_size, 1, -1), emb.unsqueeze(2)).squeeze(2)  # Shape: (batch_size, 1)
        rnd = torch.bmm(rnd.view(batch_size, 1, -1), emb.unsqueeze(2)).squeeze(2)  # Shape: (batch_size, 1)

        # Apply sigmoid and clamp to prevent NaNs
        out = self.sig(out).clamp(min=1e-7, max=1 - 1e-7)
        rnd = self.sig(rnd).clamp(min=1e-7, max=1 - 1e-7)

        # Calculate loss
        pst = -out.log().mean()   # Positive sample log-likelihood
        ngt = -(1 - rnd).log().mean()  # Negative sample log-likelihood
        
        return pst + ngt

# Step 4: Load the trained SkipGram model
model_path = "finetuned_skipgram_model.pth"
embedding_dim = 64  # Set to the same dimension used when training

# Ensure the model is moved to the GPU
mFoo = SkipGramFoo(len(updated_vocab), embedding_dim, 2).to(device)

# Load the saved model weights
try:
    mFoo.load_state_dict(torch.load(model_path), strict=False)  # Use strict=False to allow for flexible loading
except RuntimeError as e:
    print(f"Error loading state dict: {e}")

mFoo.eval()  # Set the model to evaluation mode

# Step 5: Generate embeddings for each title
def get_embeddings_for_titles(tokenized_titles, model):
    embeddings_list = []
    
    with torch.no_grad():  # Disable gradient calculations for faster performance
        for tokens in tokenized_titles:
            if len(tokens) > 0:
                # Move the tokens to the GPU
                token_tensor = torch.LongTensor(tokens).to(device)
                
                # Get the embeddings for each token in the title
                token_embeddings = model.emb(token_tensor)  # Shape: [num_tokens, embedding_dim]
                
                # Average the token embeddings to get a single vector for the entire title
                title_embedding = token_embeddings.mean(dim=0)  # Shape: [embedding_dim]
                
                embeddings_list.append(title_embedding.cpu().numpy())  # Store the embedding as a NumPy array
            else:
                # Handle empty titles (if any)
                embeddings_list.append(torch.zeros(embedding_dim).cpu().numpy())  # Zero vector for empty titles
    
    return embeddings_list

# Generate embeddings for all the tokenized titles
embeddings = get_embeddings_for_titles(tokenized_titles, mFoo)

# Step 6: Store the embeddings
# Convert to a DataFrame for easier saving/processing later
embeddings_df = pd.DataFrame(embeddings)

# Save the embeddings to a file (optional)
embeddings_df.to_csv('title_embeddings.csv', index=False)


C:\Users\omare\AppData\Local\Temp\ipykernel_26928\793611519.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mFoo.load_state_dict(torch.load(model_path), strict=False)  

In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence  # For padding sequences
from tqdm import tqdm  # For progress bar
import wandb  # Import Weights and Biases

# Initialize W&B project
wandb.init(project="twotower_training", entity="omareweis123", name='fine_tuning_skipgram/final/10ep')

# Load the saved SkipGram model
model_save_path = "finetuned_skipgram_model.pth"
checkpoint = torch.load(model_save_path)

# Initialize the SkipGram model
skipgram_model = SkipGramFoo(121598, 64, 2).to(device)  # Ensure to send the model to the correct device
skipgram_model.load_state_dict(checkpoint['model_state_dict'])
skipgram_model.eval()  # Set the model to evaluation mode if you're not training it again

# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Define query batch size
query_batch_size = 64  # Number of queries to process in a batch

# Define number of epochs
num_epochs = 10  # Example

# Track model and hyperparameters in W&B
wandb.watch([tower_one, tower_two], log="all")  # Log gradients and model weights

# Training loop with batched queries
for epoch in range(num_epochs):
    # Group by 'query' to handle variable number of positives and negatives per query
    query_groups = list(results.groupby('query'))

    # Iterate through the dataset in batches of 'query_batch_size' queries
    for q_batch_start in tqdm(range(0, len(query_groups), query_batch_size), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        query_batch = query_groups[q_batch_start:q_batch_start + query_batch_size]

        all_anchor_embeddings = []
        all_positive_embeddings = []
        all_negative_embeddings = []

        # Process each query group in the current batch
        for query, group in query_batch:
            # Tokenize the queries, passage_text, and negative_sample using your updated_vocab or reverse_vocab
            query_tokens = tokenize_titles(group['query'].tolist(), reverse_vocab)
            positive_tokens = tokenize_titles(group['passage_text'].tolist(), reverse_vocab)
            negative_tokens = tokenize_titles(group['negative_sample'].tolist(), reverse_vocab)

            # Get embeddings for each group
            anchor_embeddings = get_embeddings_for_titles(query_tokens, skipgram_model)
            positive_embeddings = get_embeddings_for_titles(positive_tokens, skipgram_model)
            negative_embeddings = get_embeddings_for_titles(negative_tokens, skipgram_model)

            # Append the embeddings to the lists
            all_anchor_embeddings.append(torch.tensor(anchor_embeddings))
            all_positive_embeddings.append(torch.tensor(positive_embeddings))
            all_negative_embeddings.append(torch.tensor(negative_embeddings))

        # Pad sequences to the same length
        anchor_batch = pad_sequence(all_anchor_embeddings, batch_first=True).to(device)
        positive_batch = pad_sequence(all_positive_embeddings, batch_first=True).to(device)
        negative_batch = pad_sequence(all_negative_embeddings, batch_first=True).to(device)

        # Forward pass through the two towers (anchor -> TowerOne, positive/negative -> TowerTwo)
        anchor_output = tower_one(anchor_batch)  # anchor_batch is now 3D (batch_size, sequence_length, embedding_size)
        positive_output = tower_two(positive_batch)
        negative_output = tower_two(negative_batch)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

        # Log the loss to W&B
        wandb.log({"epoch": epoch + 1, "triplet_loss": triplet_loss.item()})

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {triplet_loss.item()}")

    # Optionally, save a model checkpoint after every epoch and log it to W&B
    torch.save({
        'model_state_dict': skipgram_model.state_dict(),
        'tower_one_state_dict': tower_one.state_dict(),
        'tower_two_state_dict': tower_two.state_dict(),
    }, f"model_checkpoint_epoch_{epoch+1}.pth")

    wandb.finish

print("Training complete.")


C:\Users\omare\AppData\Local\Temp\ipykernel_26928\3823745590.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path)
Epoch 1/10: 100%|█

Epoch 1/10 completed. Loss: 0.6824356913566589


Epoch 2/10: 100%|██████████| 1287/1287 [07:28<00:00,  2.87it/s]


Epoch 2/10 completed. Loss: 0.6674662232398987


Epoch 3/10: 100%|██████████| 1287/1287 [07:18<00:00,  2.94it/s]


...

Epoch 8/10 completed. Loss: 0.6206641793251038


Epoch 9/10: 100%|██████████| 1287/1287 [07:27<00:00,  2.87it/s]


Epoch 9/10 completed. Loss: 0.5360973477363586


Epoch 10/10: 100%|██████████| 1287/1287 [07:57<00:00,  2.69it/s]


Epoch 10/10 completed. Loss: 0.47312402725219727
Training complete.


In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # For progress bar

# Load the saved SkipGram model
model_save_path = "finetuned_skipgram_model.pth"
checkpoint = torch.load(model_save_path)

# Initialize the SkipGram model
skipgram_model = SkipGramFoo(121598, 64, 2).to(device)  # Ensure to send the model to the correct device
skipgram_model.load_state_dict(checkpoint['model_state_dict'])
skipgram_model.eval()  # Set the model to evaluation mode if you're not training it again

# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Training with query-based batching
for epoch in range(num_epochs):
    # Group by 'query' to handle variable number of positives and negatives per query
    query_groups = results.groupby('query')
    
    for query, group in tqdm(query_groups, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Tokenize the queries, passage_text, and negative_sample using your updated_vocab or reverse_vocab
        query_tokens = tokenize_titles(group['query'].tolist(), reverse_vocab)
        positive_tokens = tokenize_titles(group['passage_text'].tolist(), reverse_vocab)
        negative_tokens = tokenize_titles(group['negative_sample'].tolist(), reverse_vocab)

        # Create tensors for inputs, ensuring they're on the correct device
        anchor_embeddings = get_embeddings_for_titles(query_tokens, skipgram_model)
        positive_embeddings = get_embeddings_for_titles(positive_tokens, skipgram_model)
        negative_embeddings = get_embeddings_for_titles(negative_tokens, skipgram_model)

        # Move embeddings to device and add sequence dimension
        anchor_embeddings = torch.tensor(anchor_embeddings).unsqueeze(1).to(device)  # (batch_size, 1, embedding_size)
        positive_embeddings = torch.tensor(positive_embeddings).unsqueeze(1).to(device)  # Same for positive
        negative_embeddings = torch.tensor(negative_embeddings).unsqueeze(1).to(device)  # Same for negative

        # Forward pass through the two towers (anchor -> TowerOne, positive/negative -> TowerTwo)
        anchor_output = tower_one(anchor_embeddings)
        positive_output = tower_two(positive_embeddings)
        negative_output = tower_two(negative_embeddings)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {triplet_loss.item()}")

print("Training complete.")


C:\Users\omare\AppData\Local\Temp\ipykernel_26928\3043405766.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path)
Epoch 1/1000:   2%|

KeyboardInterrupt: 

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # For progress bar

# Load the saved SkipGram model
model_save_path = "finetuned_skipgram_model.pth"
checkpoint = torch.load(model_save_path)

# Initialize the SkipGram model
skipgram_model = SkipGramFoo(121598, 64, 2).to(device)  # Ensure to send the model to the correct device
skipgram_model.load_state_dict(checkpoint['model_state_dict'])
skipgram_model.eval()  # Set the model to evaluation mode if you're not training it again

# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Training with query-based batching
for epoch in range(num_epochs):
    # Group by 'query' to handle variable number of positives and negatives per query
    query_groups = results.groupby('query')
    
    for query, group in tqdm(query_groups, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Tokenize the queries, passage_text, and negative_sample using your updated_vocab or reverse_vocab
        query_tokens = tokenize_titles(group['query'].tolist(), reverse_vocab)
        positive_tokens = tokenize_titles(group['passage_text'].tolist(), reverse_vocab)
        negative_tokens = tokenize_titles(group['negative_sample'].tolist(), reverse_vocab)

        # Create tensors for inputs, ensuring they're on the correct device
        anchor_embeddings = get_embeddings_for_titles(query_tokens, skipgram_model)
        positive_embeddings = get_embeddings_for_titles(positive_tokens, skipgram_model)
        negative_embeddings = get_embeddings_for_titles(negative_tokens, skipgram_model)

        # Move embeddings to device
        anchor_embeddings = torch.tensor(anchor_embeddings).to(device)
        positive_embeddings = torch.tensor(positive_embeddings).to(device)
        negative_embeddings = torch.tensor(negative_embeddings).to(device)

        # Forward pass through the two towers (anchor -> TowerOne, positive/negative -> TowerTwo)
        anchor_output = tower_one(anchor_embeddings)
        positive_output = tower_two(positive_embeddings)
        negative_output = tower_two(negative_embeddings)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {triplet_loss.item()}")

print("Training complete.")


C:\Users\omare\AppData\Local\Temp\ipykernel_26928\418175315.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path)
Epoch 1/1000:   0%| 

IndexError: too many indices for tensor of dimension 2

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # For progress bar

# Load the saved SkipGram model
model_save_path = "finetuned_skipgram_model.pth"
checkpoint = torch.load(model_save_path)

# Initialize the SkipGram model
skipgram_model = SkipGramFoo(121598, 64, 2)
skipgram_model.load_state_dict(checkpoint['model_state_dict'])

# Tower RNN Models
tower_one = TowerOneRNN()
tower_two = TowerTwoRNN()

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Training with query-based batching
for epoch in range(num_epochs):
    # Group by 'query' to handle variable number of positives and negatives per query
    query_groups = results.groupby('query')
    
    for query, group in tqdm(query_groups, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Anchor: query embeddings
        anchor_embeddings = skipgram_model(group['query'].tolist())
        
        # Positive samples: embedding of passage_text
        positive_embeddings = skipgram_model(group['passage_text'].tolist())
        
        # Negative samples: embedding of negative_sample
        negative_embeddings = skipgram_model(group['negative_sample'].tolist())
        
        # Forward pass through the two towers (anchor -> TowerOne, positive/negative -> TowerTwo)
        anchor_output = tower_one(anchor_embeddings)
        positive_output = tower_two(positive_embeddings)
        negative_output = tower_two(negative_embeddings)
        
        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)
        
        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {triplet_loss.item()}")

print("Training complete.")


C:\Users\omare\AppData\Local\Temp\ipykernel_26928\2370165924.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path)
Epoch 1/1000:   0%|

TypeError: SkipGramFoo.forward() missing 2 required positional arguments: 'trgs' and 'rand'